In [141]:
# Import Subsystems
include("../examples/example_functions.jl")
include("../finite_difference.jl")
include("../numerical_continuation.jl")
include("../numerical_shooting.jl")
include("../ode_solver.jl")
include("../visualisation.jl")

plot_phase_portrait_3d (generic function with 2 methods)

In [158]:
include("../numerical_continuation.jl")
par_values, conditions = continuation(predprey, [1 1], 6, "b", 0.2:0.001:0.3, discretisation="shooting", method="pa")

# Create trace
u1 = scatter(
    x = par_values,
    y = conditions[:,1],
    mode="lines",
    name="u1",
    showlegend=true
    )
u2 = scatter(
    x = par_values,
    y = conditions[:,2],
    mode="lines",
    name="u2",
    showlegend=true
    )

layout = Layout(
    xaxis_title = "parameter",
    # yaxis_type="log",
    yaxis_exponentformat="power",
    # yaxis_title = "",
    width=700, height=350,
    )

plot([u1, u2], layout)

data: [
  "scatter with fields mode, name, showlegend, type, x, and y",
  "scatter with fields mode, name, showlegend, type, x, and y"
]

layout: "layout with fields height, margin, template, width, xaxis, and yaxis"

In [145]:
include("../numerical_continuation.jl")
rng = 2:-0.1:-1
par_values, conditions = continuation(hopf2d_modified, [-1 0], 6, "beta", rng, method = "npc", discretisation="shooting")
par_values2, conditions2 = continuation(hopf2d_modified, [-1 0], 6, "beta", rng, method = "pa", discretisation="shooting")

# Create trace
u1 = scatter(
    x = par_values,
    y = conditions[:,1],
    mode="lines",
    name="u1 npc",
    showlegend=true
    )
u2 = scatter(
    x = par_values,
    y = conditions[:,2],
    mode="lines",
    name="u2 npc",
    showlegend=true
    )

layout = Layout(
    xaxis_title = "parameter",
    # yaxis_type="log",
    yaxis_exponentformat="power",
    # yaxis_title = "",
    width=700, height=350,
    )

u3 = scatter(
    x = par_values2,
    y = conditions2[:,1],
    mode="lines",
    name="u1",
    showlegend=true
    )
u4 = scatter(
    x = par_values2,
    y = conditions2[:,2],
    mode="lines",
    name="u2",
    showlegend=true
    )


p1 = plot([u1, u2], layout)
p2 = plot([u3, u4], layout)

p = [p1 p2]
relayout!(p)
p

data: [
  "scatter with fields mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields mode, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"

In [154]:
include("../examples/example_functions.jl")
include("../numerical_continuation.jl")
include("../visualisation.jl")


# Excercise 1
algebraic_pv_1, algebraic_c_1 = continuation(algebraic,[1 1],6,"c",-2:0.05:2,method = "natural_parameter_continuation",discretisation = "none",);
hopf2d_pv_1, hopf2d_c_1 = continuation(hopf2d,[-1 0],6,"beta",2:-0.05:0,method = "natural_parameter_continuation",discretisation = "shooting",);
hopf2d_mod_pv_1, hopf2d_mod_c_1 = continuation(hopf2d_modified,[-1 0],6,"beta",2:-0.05:-1,method = "natural_parameter_continuation",discretisation = "shooting",);

# Excercise 2
algebraic_pv_2, algebraic_c_2 = continuation(algebraic,[1 1],6,"c",-2:0.05:2,method = "pseudo_arclength",discretisation = "none",);
hopf2d_pv_2, hopf2d_c_2 = continuation(hopf2d,[-1 0],6,"beta",2:-0.05:0,method = "pseudo_arclength",discretisation = "shooting",);
hopf2d_mod_pv_2, hopf2d_mod_c_2 = continuation(hopf2d_modified,[-1 0],6,"beta",2:-0.05:-1,method = "pseudo_arclength",discretisation = "shooting",);

# Create traces
p1_u1 = scatter(x = algebraic_pv_1,y = algebraic_c_1[:,1],mode="lines",name="u1",showlegend=true, line=attr(color="blue"))
p1_u2 = scatter(x = algebraic_pv_1,y = algebraic_c_1[:,2],mode="lines",name="u2",showlegend=true, line=attr(color="red"))

p2_u1 = scatter(x = hopf2d_pv_1,y = hopf2d_c_1[:,1],mode="lines",name="u1",showlegend=false, line=attr(color="blue"))
p2_u2 = scatter(x = hopf2d_pv_1,y = hopf2d_c_1[:,2],mode="lines",name="u2",showlegend=false, line=attr(color="red"))

p3_u1 = scatter(x = hopf2d_mod_pv_1,y = hopf2d_mod_c_1[:,1],mode="lines",name="u1",showlegend=false, line=attr(color="blue"))
p3_u2 = scatter(x = hopf2d_mod_pv_1,y = hopf2d_mod_c_1[:,2],mode="lines",name="u2",showlegend=false, line=attr(color="red"))

p4_u1 = scatter(x = algebraic_pv_2,y = algebraic_c_2[:,1],mode="lines",name="u1",showlegend=false, line=attr(color="blue"))
p4_u2 = scatter(x = algebraic_pv_2,y = algebraic_c_2[:,2],mode="lines",name="u2",showlegend=false, line=attr(color="red"))

p5_u1 = scatter(x = hopf2d_pv_2,y = hopf2d_c_2[:,1],mode="lines",name="u1",showlegend=false, line=attr(color="blue"))
p5_u2 = scatter(x = hopf2d_pv_2,y = hopf2d_c_2[:,2],mode="lines",name="u2",showlegend=false, line=attr(color="red"))

p6_u1 = scatter(x = hopf2d_mod_pv_2,y = hopf2d_mod_c_2[:,1],mode="lines",name="u1",showlegend=false, line=attr(color="blue"))
p6_u2 = scatter(x = hopf2d_mod_pv_2,y = hopf2d_mod_c_2[:,2],mode="lines",name="u2",showlegend=false, line=attr(color="red"))



p1 = plot([p1_u1, p1_u2], Layout(title="Algebraic NPC", xaxis_title = "c", yaxis_title = ""))
p2 = plot([p2_u1, p2_u2], Layout(title="Hopf NPC", xaxis_title = "beta", yaxis_title = ""))
p3 = plot([p3_u1, p3_u2], Layout(title="Modified Hopf NPC", xaxis_title = "beta", yaxis_title = ""))
p4 = plot([p4_u1, p4_u2], Layout(title="Algebraic PA", xaxis_title = "c", yaxis_title = ""))
p5 = plot([p5_u1, p5_u2], Layout(title="Hopf PA", xaxis_title = "beta", yaxis_title = ""))
p6 = plot([p6_u1, p6_u2], Layout(title="Modified Hopf PA", xaxis_title = "beta", yaxis_title = ""))

ncm_comp_plot = [p1 p2 p3; p4 p5 p6]
relayout!(ncm_comp_plot, height=500, width=800, title_text="Numerical Continuation Method Comparison")
ncm_comp_plot

data: [
  "scatter with fields line, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, height, margin, template, title, width, xaxis1, xaxis2, xaxis3, xaxis4, xaxis5, xaxis6, yaxis1, yaxis2, yaxis3, yaxis4, yaxis5, and yaxis6"

## Workspace

In [160]:
i = 1

discretisation = (u0, par) -> algebraic(u0, c = par)

v0 = [new_par_values[i] conditions[[i], :]]
v1 = [new_par_values[i+1] conditions[[i + 1], :]]
secant = v1 - v0
v_pred = v1 + secant
sol = nlsolve(
        (v2) -> [discretisation(v2[:, 2:end], v2[1]) pseudo_arclength_eq(
            secant,
            v2,
            v_pred,
        )],
        v_pred,
    ).zero


conditions = [conditions; sol[:, 2:end]]
push!(new_par_values, sol[1])

5-element Vector{Float64}:
   -2.0
   -1.99
   -1.980002037489294
 -100.33262819918922
 -100.33262819918922

In [161]:
par_values[1]

0.2

In [98]:
par_values[end]

2.0

In [94]:
0.5 ∈ range(0,1)

LoadError: UndefVarError: ∞ not defined

In [138]:
par_values = 0:0.01:1
par_values[end]

1.0

In [139]:
pars_test = 0.5
for tsts in pars_test
    println(xor(tsts < par_values[end], tsts < par_values[1]) 
        && xor(tsts > par_values[end], tsts > par_values[1]))
end


true


In [104]:
xor(new_par_values[end] < par_values[end], new_par_values[end] > par_values[end])
xor(new_par_values[end] < par_values[end], new_par_values[end] < par_values[1])

true

In [69]:
end_function(3)


true

In [48]:
function foo(x; beta=0, alpha=1, arg...)

    return x*beta*alpha
    
end

foo (generic function with 1 method)

In [32]:
foo(3, beta=0)

0

In [51]:
function bar(x, parameter; range=0:0.1:2, arg...)

    y = foo(x; parameter=range, arg...) # doesn't work. Could it?

    return 3*y
end

bar (generic function with 2 methods)

In [55]:
bar(3, beta=1, alpha=2)

18

In [94]:
# ts = -1:0.1:2
ts = 2:-0.1:-1

2.0:-0.1:-1.0